<a href="https://colab.research.google.com/github/lala991204/DL-self-study/blob/master/tensorflow/4_7_distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
from google.colab.patches import cv2_imshow
from tqdm import tqdm

파라미터 개수가 많은 큰 모델이 선생님이 되어 크기가 작은 모델을 가르치는 개념으로 Knowledge Distillation이라 부름. 큰 모델의 예측과 작은 모델의 예측의 오차(distillation)와 작은 모델의 손실함수(student loss)를 줄여 나가는 방향으로 작은 모델의 파라미터를 최적화함.

In [ ]:
# @title 파라미터 설정
t_ephoc = 5      # @param {type:"slider", min:1, max:100, step:1}
s_ephoc = 10     # @param {type:"slider", min:1, max:100, step:1}
learning_rate = 0.01
batch_size = 64  # @param [32, 64, 128, 256] {type:"raw"} 
temperature = 3  # @param {type:"slider", min:1, max:10, step:1}
alpha = 0.5      # @param {type:"slider", min:0.1, max:0.9, step:0.1}

In [ ]:
# mnist dataset 가져오기
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype("float32")/255.0
x_train = np.reshape(x_train, (-1, 28, 28, 1))     # 배치 사이즈가 들어갈 축 추가

x_test = x_test.astype("float32")/255.0
x_test = np.reshape(x_test, (-1, 28, 28, 1))


# teacher model(비교적 복잡한 모델 구성)
i = tf.keras.Input(shape=(28, 28, 1))
out = tf.keras.layers.Conv2D(256, (3, 3), strides=(2, 2), padding='same')(i)
out = tf.keras.layers.LeakyReLU(alpha=0.2)(out)
out = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same")(out)
out = tf.keras.layers.Conv2D(512, (3, 3), strides=(2, 2), padding='same')(out)
out = tf.keras.layers.Flatten()(out)
out = tf.keras.layers.Dense(10)(out)
t_model = tf.keras.Model(inputs=[i], outputs=[out])

t_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 14, 14, 256)       2560      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 14, 14, 256)       0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 256)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 512)         1180160   
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                             

In [ ]:
# student model(단순한 구조)
i = tf.keras.Input(shape=(28, 28, 1)) 
out = tf.keras.layers.Flatten()(i)
out = tf.keras.layers.Dense(28)(out)
out = tf.keras.layers.Dense(10)(out)

s_model_1 = tf.keras.Model(inputs=[i], outputs=[out])
s_model_2 = tf.keras.models.clone_model(s_model_1)       # 성능 비교를 위해 모델 복제

s_model_1.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_1 (Dense)             (None, 28)                21980     
                                                                 
 dense_2 (Dense)             (None, 10)                290       
                                                                 
Total params: 22,270
Trainable params: 22,270
Non-trainable params: 0
_________________________________________________________________


파라미터가 teacher model의 약 1/70에 불과함

In [ ]:
# teacher model
t_model.compile(tf.keras.optimizers.Adam(learning_rate),
                tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# student model(distillation 적용)
s_model_1.compile(tf.keras.optimizers.Adam(learning_rate),
                  tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# 비교 model(distillation 미적용)
s_model_2.compile(tf.keras.optimizers.Adam(learning_rate),
                  tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])


t_model.fit(x_train, y_train, batch_size = batch_size, epochs = t_ephoc)

Epoch 1/5
938/938 [==============================] - 722s 769ms/step - loss: 0.5496 - sparse_categorical_accuracy: 0.9208
Epoch 2/5
938/938 [==============================] - 596s 635ms/step - loss: 13.8130 - sparse_categorical_accuracy: 0.9372
Epoch 3/5
938/938 [==============================] - 545s 581ms/step - loss: 4.3400 - sparse_categorical_accuracy: 0.9617
Epoch 4/5
938/938 [==============================] - 547s 583ms/step - loss: 7.9766 - sparse_categorical_accuracy: 0.9581
Epoch 5/5
938/938 [==============================] - 545s 581ms/step - loss: 9.8430 - sparse_categorical_accuracy: 0.9635


teacher model의 경우 3 epoch만에 약 96%의 정확도 보임.

In [ ]:
## 다음은 Knowledge Distillation 학습에 필요한 loss들이다.
# student 손실함수
s_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# distillation 손실함수
d_loss = tf.keras.losses.KLDivergence()    

x_train.shape

(60000, 28, 28, 1)

참고로, KLDivergence는 서로 다른 두 개의 확률분포를 비교하여 유사성을 측정하는 지표이며, 서로 유사할수록 값이 작아짐.

In [ ]:
batch_count = x_train.shape[0]//batch_size         # 총 배치의 개수
opt = tf.keras.optimizers.Adam(learning_rate)
for e in range(s_ephoc):
    for _ in range(batch_count):         # 배치별로 각각의 loss 계산
        batch_num = np.random.randint(0, x_train.shape[0], size=batch_size)
        t_pred = t_model.predict(x_train[batch_num])

        with tf.GradientTape() as tape:
            s_pred_1 = s_model_1(x_train[batch_num])
            student_loss = s_loss(y_train[batch_num], s_pred_1)
            distillation_loss = d_loss(
                tf.nn.softmax(t_pred / temperature, axis=1),
                tf.nn.softmax(s_pred_1 / temperature, axis=1),
            )
            loss = alpha * student_loss + (1 - alpha) * distillation_loss
        
        vars = s_model_1.trainable_variables
        grad = tape.gradient(loss, vars)
        opt.apply_gradients(zip(grad, vars))

        with tf.GradientTape() as tape:
            s_pred_2 = s_model_2(x_train[batch_num])
            student_loss = s_loss(y_train[batch_num], s_pred_2)
        vars = s_model_2.trainable_variables
        grad = tape.gradient(student_loss, vars)
        opt.apply_gradients(zip(grad, vars))

    print("epoch {}".format(e)) 
    print("선생님께 배운 경우") 
    s_model_1.evaluate(x_test, y_test)
    print("혼자 공부한 경우")
    s_model_2.evaluate(x_test, y_test)
    print("\n")      

epoch 0
선생님께 배운 경우
313/313 [==============================] - 1s 1ms/step - loss: 0.4531 - sparse_categorical_accuracy: 0.9078
혼자 공부한 경우
313/313 [==============================] - 1s 1ms/step - loss: 0.3678 - sparse_categorical_accuracy: 0.8989


epoch 1
선생님께 배운 경우
313/313 [==============================] - 0s 1ms/step - loss: 0.4473 - sparse_categorical_accuracy: 0.9102
혼자 공부한 경우
313/313 [==============================] - 0s 1ms/step - loss: 0.3301 - sparse_categorical_accuracy: 0.9100


epoch 2
선생님께 배운 경우
313/313 [==============================] - 0s 2ms/step - loss: 0.5328 - sparse_categorical_accuracy: 0.8983
혼자 공부한 경우
313/313 [==============================] - 0s 1ms/step - loss: 0.3676 - sparse_categorical_accuracy: 0.8969


epoch 3
선생님께 배운 경우
313/313 [==============================] - 0s 2ms/step - loss: 0.4953 - sparse_categorical_accuracy: 0.9028
혼자 공부한 경우
313/313 [==============================] - 0s 1ms/step - loss: 0.3572 - sparse_categorical_accuracy: 0.9034


epoch 4
선생님께